In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score,accuracy_score,log_loss,r2_score
from sklearn.model_selection import train_test_split,KFold , GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer ,make_column_selector
from sklearn.pipeline import Pipeline

In [4]:
train = pd.read_csv('train.csv').set_index('id')

In [6]:
train.head()

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
id,,,,,,,,,,,,
0,100.0,0.841611,10.0,4.8,20.612526,11.08810,2.766,1.732,0.860,0.496070,0.91457,6.0
1,100.0,7.558488,10.0,4.8,20.298893,12.04083,2.755,1.631,0.910,0.492719,0.71760,6.5
2,76.0,8.885992,15.6,5.6,33.739258,12.08630,2.828,1.788,0.864,0.481478,1.50633,2.5
3,100.0,8.795296,10.0,4.8,20.213349,10.94850,2.648,1.626,0.936,0.489272,0.78937,6.0
4,116.0,9.577996,11.6,4.8,24.988133,11.82448,2.766,1.682,0.896,0.492736,1.86481,6.0


In [7]:
X = train.drop('Hardness',axis=1)
y = train['Hardness']

In [28]:
dtr = DecisionTreeRegressor(random_state=24)
dtr.fit(X,y)


DecisionTreeRegressor(random_state=24)

In [35]:

dtr =DecisionTreeRegressor(random_state=24)

params = {'min_samples_split':[2,10,25,10],
         'min_samples_leaf':[1,2,5,7,10,25,50],
         'max_depth':[None,4,6]}
kfold = KFold(n_splits=5,shuffle=True,random_state=24)

gcv = GridSearchCV(dtr,param_grid=params,cv=kfold,scoring='r2',verbose=2)
gcv.fit(X,y)

Fitting 5 folds for each of 84 candidates, totalling 420 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=25; total time=   0.

GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=DecisionTreeRegressor(random_state=24),
             param_grid={'max_depth': [None, 4, 6],
                         'min_samples_leaf': [1, 2, 5, 7, 10, 25, 50],
                         'min_samples_split': [2, 10, 25, 10]},
             scoring='r2', verbose=2)

In [36]:
print(gcv.best_params_)
print(gcv.best_score_)

{'max_depth': None, 'min_samples_leaf': 50, 'min_samples_split': 2}
0.39799384830214196


In [37]:
best_tree = gcv.best_estimator_

In [38]:
X_test  = pd.read_csv('test.csv',index_col = 0)

In [41]:
y_pred = best_tree.predict(X_test)

In [43]:
submit  = pd.read_csv('sample_submission.csv')

In [44]:
submit['Hardness'] = y_pred

In [46]:
submit.to_csv('ss_8_nov_1_lr.csv',index=False)